Time-series forecasting to forecast store sales on data from Corporación Favorita, a large Ecuadorian-based grocery retailer https://www.kaggle.com/competitions/store-sales-time-series-forecasting


File Descriptions and Data Field Information:

train.csv

    The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
    store_nbr identifies the store at which the products are sold.
    family identifies the type of product sold.
    sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
    onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.

test.csv

    The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
    The dates in the test data are for the 15 days after the last date in the training data.

sample_submission.csv

    A sample submission file in the correct format.

stores.csv

    Store metadata, including city, state, type, and cluster.
    cluster is a grouping of similar stores.

oil.csv

    Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

holidays_events.csv

    Holidays and Events, with metadata
    NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
    Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

Additional Notes

    Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
    A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.


In [75]:
import os
import datetime

import IPython
import IPython.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from darts import TimeSeries


In [76]:
train = pd.read_csv("input/train.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [77]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [78]:
train.date = pd.to_datetime(train.date)

In [79]:
test = pd.read_csv("input/test.csv")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [80]:
test.date = pd.to_datetime(test.date)

In [81]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [82]:
holidays_events = pd.read_csv("input/holidays_events.csv")
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [83]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [84]:
holidays_events.date = pd.to_datetime(holidays_events.date)


In [85]:
oil = pd.read_csv("input/oil.csv")
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [86]:
oil.date =  pd.to_datetime(oil.date)

In [87]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [88]:
stores = pd.read_csv("input/stores.csv")
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [89]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [90]:
transactions = pd.read_csv("input/transactions.csv")
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [91]:
transactions.date =  pd.to_datetime(transactions.date)

In [92]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


### Check missing values

In [93]:
print((train.date.max() - train.date.min()).days + 1) # 1688
train.nunique() # 1684
# There are 4 missing dates in train data 


1688


id             3000888
date              1684
store_nbr           54
family              33
sales           379610
onpromotion        362
dtype: int64

In [94]:
print((test.date.max() - test.date.min()).days + 1) # 16
test.nunique() # 16
# There are no missing dates in test data 


16


id             28512
date              16
store_nbr         54
family            33
onpromotion      212
dtype: int64

In [95]:
dates_train = train.date.unique() 
all_dates = pd.date_range(start= train.date.min(), end=train.date.max())
all_dates[~all_dates.isin(dates_train)]

#All 4 missing dates are on Christmas 25.12 - stores are probably closed that day


DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)

We need to add missing dates for each of 54 stores (store_nbr column) and each of 33 products (family column) 

In [96]:
idxs_train = pd.MultiIndex.from_product([all_dates, train.store_nbr.unique(), train.family.unique()], names=["date", "store_nbr", "family"])
train = train.set_index(["date", "store_nbr", "family"]).reindex(idxs_train).reset_index()

In [97]:
train["id"] = train.index

In [98]:
train.fillna(0., inplace=True)

In [99]:
transactions.describe() # no transactions equal to 0 

,date,store_nbr,transactions
count,83488,83488.000000,83488.000000
mean,2015-05-20 16:07:40.866232064,26.939237,1694.602158
min,2013-01-01 00:00:00,1.000000,5.000000
25%,2014-03-27 00:00:00,13.000000,1046.000000
50%,2015-06-08 00:00:00,27.000000,1393.000000
75%,2016-07-14 06:00:00,40.000000,2079.000000
max,2017-08-15 00:00:00,54.000000,8359.000000
std,NaN,15.608204,963.286644


In [100]:
sales_store = train.groupby(["date", "store_nbr"]).sales.sum().reset_index()

In [ ]:
transactions_sales = transactions.merge(sales_store, on = ["date", "store_nbr"], how = "right")

In [119]:
transactions_sales[(transactions_sales.sales != 0) & (transactions_sales.transactions.isna())]
# there are 118 rows with NaN transactions and non zero sales

,date,store_nbr,transactions,sales
9135,2013-06-19,10,NaN,3802.291998
9160,2013-06-19,35,NaN,1699.048000
9168,2013-06-19,43,NaN,4642.495001
9179,2013-06-19,54,NaN,2977.852000
19795,2014-01-02,32,NaN,3146.146702
...,...,...,...,...
59342,2016-01-04,51,NaN,28280.580970
59344,2016-01-04,53,NaN,8702.973100
59345,2016-01-04,54,NaN,8711.512998
73716,2016-09-27,7,NaN,19783.335000


In [ ]:
transactions_sales[(transactions_sales.sales == 0) & (transactions_sales.transactions.isna())]
# there are 7546 rows with NaN transactions and zero sales

,date,store_nbr,transactions,sales
0,2013-01-01,1,NaN,0.0
1,2013-01-01,2,NaN,0.0
2,2013-01-01,3,NaN,0.0
3,2013-01-01,4,NaN,0.0
4,2013-01-01,5,NaN,0.0
...,...,...,...,...
84561,2017-04-15,52,NaN,0.0
84615,2017-04-16,52,NaN,0.0
84669,2017-04-17,52,NaN,0.0
84723,2017-04-18,52,NaN,0.0


In [ ]:
transactions_sales.loc[(transactions_sales.sales == 0) & (transactions_sales.transactions.isna()), 'transactions'] = 0
interpolated_transactions = transactions_sales.groupby('store_nbr', group_keys=False).transactions.apply(lambda x: x.interpolate(limit_direction = 'both'))
transactions_sales['transactions'] = interpolated_transactions

In [223]:
missings_oil = pd.date_range(train.date.min(), test.date.max()).difference(oil.date)
# 486 missing dates with oil prices
missings_oil.weekday.value_counts()
# All missings are on weekends - saturdays and sundays

5    243
6    243
Name: count, dtype: int64

In [ ]:
# Adding missing dates and filling values using linear interpolation 
oil = oil.merge(pd.DataFrame(missings_oil, columns=["date"]), on = "date", how = "outer")
oil["dcoilwtico"] = oil.dcoilwtico.interpolate(limit_direction="both")

In [236]:
holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
